In [1]:
import sklearn
import scipy
from pycaret.regression import *
import botorch
import torch
import gpytorch
import pandas as pd
import numpy as np

# Linear Regression com normalização

In [2]:
df=pd.DataFrame([['AISI_1045', 0.25, 1000, -1, 5, 209.3996337890626, 553.0, 600.0,0.0134, 0.234, 1.0],
                          ['AISI_4140', 0.25, 1000, -4, 5, 1147.44189453125, 595.0, 580.0,0.023, 0.133, 1.03],
                          ['AISI_4340', 0.25, 1000, -4, 5, 584.8319091796877, 792.0, 510.0,0.014, 1.03, 0.26],
                          ['AISI_52100', 0.25, 1000, -4, 5, 253.8834747314454, 774.78, 134.0,0.0018, 0.37, 3.171],
                          ['AISI_304', 0.25, 1000, -4, 5, 739.4136962890625, 310.0, 1000.0,0.07, 0.65, 1.0],
                          ['AISI_1020', 0.25, 1000, -4, 5, 80.56342620849611, 213.0, 53.0,0.0055, 0.34, 0.81],],
              columns=["material","depth","vc","rake_angle","clearance_angle","expected_cut_force",
                        "A","B","C","n","m"])
df

,material,depth,vc,rake_angle,clearance_angle,expected_cut_force,A,B,C,n,m
0,AISI_1045,0.25,1000,-1,5,209.399634,553.00,600.0,0.0134,0.234,1.000
1,AISI_4140,0.25,1000,-4,5,1147.441895,595.00,580.0,0.0230,0.133,1.030
2,AISI_4340,0.25,1000,-4,5,584.831909,792.00,510.0,0.0140,1.030,0.260
3,AISI_52100,0.25,1000,-4,5,253.883475,774.78,134.0,0.0018,0.370,3.171
4,AISI_304,0.25,1000,-4,5,739.413696,310.00,1000.0,0.0700,0.650,1.000
5,AISI_1020,0.25,1000,-4,5,80.563426,213.00,53.0,0.0055,0.340,0.810


In [3]:
def build_input(controlable_input, fixed_input,construct_dict):
    final_input_df=pd.DataFrame()
    for input in controlable_input:
        for counter,parameter in enumerate(construct_dict.keys()):
            fixed_input[parameter]=input[counter]
        final_input_df=pd.concat([final_input_df,fixed_input])
    return final_input_df

In [4]:
construct_dict={"rake_angle":{"sequence_len":4,"max_bound":10,"lower_bound":-10,"type":"int"},
                "vc":{"sequence_len":10,"max_bound":500,"lower_bound":30,"type":"int"},
                #"depth":{"sequence_len":4,"max_bound":2,"lower_bound":0.1,"type":"float"}
                }

In [5]:
degree_to_radian=lambda x: x*np.pi/180
radians_to_degree=lambda x: x*180/np.pi

# Minimização da força de corte

In [6]:
lasso_v0=load_model("models/lasso_v0")

Transformation Pipeline and Model Successfully Loaded


In [7]:
def objective_func(x,iloc_=0):
    #x -> [[rake_angle,vc]] 
    x=[x]
    inputs=build_input(x,df.iloc[[iloc_]],construct_dict)
    cut_force=(predict_model(lasso_v0,inputs).drop_duplicates()["prediction_label"].values)
    return cut_force[0]

## método de minimização

In [8]:
from scipy.optimize import minimize

degree_to_radian=lambda x: x*np.pi/180
radians_to_degree=lambda x: x*180/np.pi

bounds=[(degree_to_radian(-15),degree_to_radian(15)),(3000,5000)]

output=minimize(objective_func,x0=[0,4500],method="Powell",bounds=bounds)
output

 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 223.38559519400474
       x: [-2.618e-01  3.000e+03]
     nit: 2
   direc: [[ 1.000e+00  0.000e+00]
           [ 1.424e-05  8.161e-02]]
    nfev: 136

In [9]:
df

,material,depth,vc,rake_angle,clearance_angle,expected_cut_force,A,B,C,n,m
0,AISI_1045,0.25,1000,-1,5,209.399634,553.00,600.0,0.0134,0.234,1.000
1,AISI_4140,0.25,1000,-4,5,1147.441895,595.00,580.0,0.0230,0.133,1.030
2,AISI_4340,0.25,1000,-4,5,584.831909,792.00,510.0,0.0140,1.030,0.260
3,AISI_52100,0.25,1000,-4,5,253.883475,774.78,134.0,0.0018,0.370,3.171
4,AISI_304,0.25,1000,-4,5,739.413696,310.00,1000.0,0.0700,0.650,1.000
5,AISI_1020,0.25,1000,-4,5,80.563426,213.00,53.0,0.0055,0.340,0.810


In [10]:
for param,value in zip(["rake_angle","vc"],output["x"]):
    if param == "rake_angle":
        value=radians_to_degree(value)
    print("{} : {:.4f}".format(param,value))

rake_angle : -14.9976
vc : 3000.0002


In [12]:
from scipy.optimize import minimize

degree_to_radian=lambda x: x*np.pi/180
radians_to_degree=lambda x: x*180/np.pi

final_df=pd.DataFrame()
step_history = []

def save_step(xk):
    step_history.append({
        'rake_angle': xk[0],
        'vc': xk[1],
        'material':material,
    })

bounds=[(-15,15),(3000,5000)]
for counter,material in enumerate(df["material"]):
    def objective_func(x,iloc_=counter):
        #x -> [[rake_angle,vc]] 
        x=[x]
        inputs=build_input(x,df.iloc[[iloc_]],construct_dict)
        cut_force=(predict_model(lasso_v0,inputs).drop_duplicates()["prediction_label"].values)
        return cut_force[0]

    output=minimize(objective_func,x0=[15,4500],method="Powell",bounds=bounds,callback=save_step)
    temp_df=df.iloc[[counter]].assign(rake_angle=output.x[0],vc=output.x[1],best_cutting_force=output.fun,status=output.success)

    final_df=pd.concat([temp_df,final_df])

In [13]:
final_df

,material,depth,vc,rake_angle,clearance_angle,expected_cut_force,A,B,C,n,m,best_cutting_force,status
5,AISI_1020,0.25,3000.000273,3.444633,5,80.563426,213.00,53.0,0.0055,0.340,0.810,66.379543,True
4,AISI_304,0.25,3000.000196,14.999965,5,739.413696,310.00,1000.0,0.0700,0.650,1.000,673.085582,True
3,AISI_52100,0.25,3000.005795,7.582842,5,253.883475,774.78,134.0,0.0018,0.370,3.171,281.150463,True
2,AISI_4340,0.25,3000.002961,-13.355016,5,584.831909,792.00,510.0,0.0140,1.030,0.260,555.688119,True
1,AISI_4140,0.25,3000.000770,-3.105183,5,1147.441895,595.00,580.0,0.0230,0.133,1.030,1135.620653,True
0,AISI_1045,0.25,3000.000802,-0.721453,5,209.399634,553.00,600.0,0.0134,0.234,1.000,223.340922,True


In [14]:
step_history=pd.DataFrame.from_dict(step_history).reset_index()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Supondo que temos o step_history já preenchido
# Exemplo de step_history
# step_history = pd.DataFrame({
#     "rake_angle": [...],
#     "vc": [...],
#     "material": [...],
#     "index": [...]
# })

# Obter os materiais únicos
materials = step_history["material"].unique()

# Função para gerar os contornos da função objetivo
def objective_func(x, y, material):
    # Simula a avaliação da função objetivo para visualização
    inputs = build_input([[x, y]], df[df["material"] == material], construct_dict)
    cut_force = predict_model(lasso_v0, inputs).drop_duplicates()["prediction_label"].values
    return cut_force[0]

# Gerar os plots
for material in materials:
    # Dados do material atual
    material_data = step_history[step_history["material"] == material]
    x_steps = material_data["rake_angle"].values
    y_steps = material_data["vc"].values

    # Geração do grid para o contour
    x_min, x_max = -15, 15
    y_min, y_max = 3000, 5000
    x = np.linspace(x_min, x_max, 25)
    y = np.linspace(y_min, y_max, 25)
    X, Y = np.meshgrid(x, y)
    Z = np.array([[objective_func(xi, yi, material) for xi, yi in zip(x_row, y_row)] for x_row, y_row in zip(X, Y)])

    # Criar o plot
    plt.figure(figsize=(8, 6))
    plt.contourf(X, Y, Z, levels=50, cmap='viridis')
    plt.colorbar(label="Cutting Force")
    plt.plot(x_steps, y_steps, '-o', color='red', label="Optimization Path")
    plt.scatter([15], [4500], color='blue', label="Initial Point")
    plt.xlabel("Rake Angle")
    plt.ylabel("VC")
    plt.title(f"Contour Plot for Material: {material}")
    plt.legend()
    plt.show()